In [37]:
include("/Users/00eila01/01_Arbete/UTBILDNING/KEXET/Numerical-Solutions-and-Parameter-Sensitivity-of-the-Lorentz-System-Degree-Project-in-Technology/Numerical-Solvers/Solvers.jl")
include("/Users/00eila01/01_Arbete/UTBILDNING/KEXET/Numerical-Solutions-and-Parameter-Sensitivity-of-the-Lorentz-System-Degree-Project-in-Technology/Numerical-Solvers/UsefulFunctions.jl")
include("/Users/00eila01/01_Arbete/UTBILDNING/KEXET/Numerical-Solutions-and-Parameter-Sensitivity-of-the-Lorentz-System-Degree-Project-in-Technology/Numerical-Solvers/PointGenerators.jl")
include("/Users/00eila01/01_Arbete/UTBILDNING/KEXET/Numerical-Solutions-and-Parameter-Sensitivity-of-the-Lorentz-System-Degree-Project-in-Technology/Numerical-Solvers/PlotFunctions.jl")

PlotPipPlot (generic function with 1 method)

In [40]:
using DifferentialEquations
using FileIO, JLD2

In [4]:
function CorrelationMatrix_Initpos(p,Δt,N,initial_points,Solver,center)
    M = zeros(length(initial_points[1,:]), N+1)
    for i = 1:length(initial_points[1,:])
        x₀ = initial_points[:,i]
        X = []
        push!(X,LorenzSolutionFixedTimeStep(p, Δt, N, x₀, Solver))
        push!(X,LorenzSolutionFixedTimeStep(p, Δt, N, center, Solver))
        for j = 1:N+1
            M[i,j] = Correlation(X[1][:,j], X[2][:,j])
        end
    end
    return M
end

CorrelationMatrix_Initpos (generic function with 1 method)

In [5]:
# initial conditions
x₀ = [0.0; 
     1.0; 
     0.0]
θ = 10; μ = 28; β = 8/3;
p=[θ,μ,β]
t = 5
Δt = 10^(-4)

0.0001

In [20]:
epsilon_vals = [10^(-10), 10^(-9), 10^(-8), 10^(-7), 10^(-6), 10^(-5), 0.0001, 0.001, 0.01, 0.1, 1]

11-element Vector{Float64}:
 1.0e-10
 1.0e-9
 1.0e-8
 1.0000000000000001e-7
 1.0e-6
 1.0e-5
 0.0001
 0.001
 0.01
 0.1
 1.0

In [24]:
initial_vals = PointsOnSphere(100, 10^(-7), x₀')

3×100 Matrix{Float64}:
 -7.54563e-9  -3.68993e-8   6.26231e-8  …  -7.71432e-8  6.02763e-8
  1.0          1.0          1.0             1.0         1.0
 -2.40644e-8   3.75993e-8  -6.75957e-8      4.41307e-8  5.71557e-8

In [77]:
length(initial_vals[1,:])

100

In [25]:
CorrelationMatrix_Initpos(p,Δt,Integer(40/Δt),initial_vals,RK4,x₀)

100×400001 Matrix{Float64}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  0.976531  0.976592  0.976652
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.707728  0.707444  0.70716
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.831817  0.831871  0.831925
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.949001  0.94908   0.949158
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.633667  0.633294  0.632922
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  0.999094  0.999086  0.999078
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.621507  0.621437  0.621366
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.984199  0.984201  0.984203
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.983193  0.98319   0.983187
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.622713  0.622817  0.622921
 ⋮                        ⋮                   ⋱                      ⋮
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.742312  0.742103  0.741893
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [26]:
N=Integer(40/Δt)
Mavg = zeros(length(epsilon_vals),N+1)
l=100
Random.seed!(3) #???

100

In [27]:
for k = 1:length(epsilon_vals)
    print(k)
    initial_vals=PointsOnSphere(l, epsilon_vals[k], x₀')
    M = CorrelationMatrix_Initpos(p,Δt,N,initial_vals,RK4,x₀);
    Mavg[k,:] = sum(M,dims=1)/l
end

12

34

56

78

910

11

In [38]:
PlotPipPlot(Mavg,Δt,clim=(0.6,1),title="Perturbations of starting position (0,1,0)")
savefig("plots/perturbations_initialpos.png")

"/Users/00eila01/01_Arbete/UTBILDNING/KEXET/Numerical-Solutions-and-Parameter-Sensitivity-of-the-Lorentz-System-Degree-Project-in-Technology/Sensitivity-Analysis/plots/perturbations_initialpos.png"

In [41]:
FileIO.save(raw"perturbation_initialpos.jld2","Mavg",Mavg)